<a href="https://colab.research.google.com/github/Joongeun/MIT-Internship-2024/blob/main/merge_science_feedback_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prompt 1 will not be used for Science Feedback because the merged labels are not accurate for credibility scores (e.x.: score of "0" means "Neutral" but with prompt 1, it would be a negative label)
import pandas as pd
import re
from pprint import pprint
import csv

df = pd.read_csv('science_feedback_data.csv', index_col=False)

#Merging verdicts and credibility scores
for ind, i in enumerate(df['verdict']):
    p = 0
    try:
        i = float(i)
        if i >= -2 and i < -1:
            p = 1
        elif i >= -1 and i < 0:
            p = 2
        elif i == 0:
            p = 3
        elif i > 0 and i <= 1:
            p = 4
        elif i > 1 and i <=2:
            p = 5
    except:
        i = i.lower()
    if i in ["accurate", "correct"] or p == 5:
        df.loc[ind, 'verdict'] = "5"
    elif i in ["mostly correct", "mostly accurate", "correct but"] or p == 4:
        df.loc[ind, 'verdict'] = "4"
    elif i in ["lacks context", "imprecise", "partially correct"] or p == 3:
        df.loc[ind, 'verdict'] = "3"
    elif i in ["unsupported", "misleading"] or p == 2:
        df.loc[ind, 'verdict'] = "2"
    elif i in ["inaccurate", "incorrect", "flawed reasoning"] or p == 1:
        df.loc[ind, 'verdict'] = "1"

#Merging claims and headlines to input to the model
inp = []
for ind, i in enumerate(df["claim"]):
    if type(i) == str:
        inp.append(i)
    else:
        inp.append(df["headline"][ind])

df["input_claims"] = inp
df.to_csv("science_feedback_merged_prompt2.csv", index=False)

In [ ]:
#Only use the input claims/headlines and verdicts to finetune model
df = pd.read_csv('science_feedback_merged_prompt2.csv', index_col=False)
df = df[df.columns[[10, 1]]]
df = df.rename(columns={"verdict": "labels"})
df.to_csv("science_feedback_finetune_data.csv", index=False)